In [1]:
import random
import seaborn as sns

# Import required for random matrix generation
import scipy.stats as stats
import scipy.sparse as sparse
from docplex.cp.utils_visu import display

from qiskit import *

In [24]:
import numpy as np
import pandas as pd
import re
import math
from qiskit import QuantumCircuit
from qiskit.quantum_info import PauliList
from qiskit_aer.primitives import Estimator, Sampler
from qiskit.circuit import Parameter

from circuit_knitting.cutting import (
    partition_problem,
    generate_cutting_experiments,
    reconstruct_expectation_values,
)

from circuit_knitting.cutting.instructions import CutWire, Move
from circuit_knitting.cutting import cut_wires, expand_observables

In [20]:
from qiskit_optimization.applications import Maxcut, Tsp, GraphPartition

# QP specific imports
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo, LinearInequalityToPenalty

# QAOA and circuit cutting specific imports
from qiskit.circuit.library import QAOAAnsatz
from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires

/var/folders/64/rp2jr10901n6vdvhsbn1fz7c0000gn/T/ipykernel_57275/193780525.py:9: DeprecationWarning: The package namespace circuit_knitting_toolbox is deprecated and will be removed no sooner than Circuit Knitting Toolbox 0.4.0. Use namespace circuit_knitting instead.
  from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires
/var/folders/64/rp2jr10901n6vdvhsbn1fz7c0000gn/T/ipykernel_57275/193780525.py:9: DeprecationWarning: The package namespace circuit_knitting_toolbox.circuit_cutting is deprecated and will be removed no sooner than Circuit Knitting Toolbox 0.4.0. Use namespace circuit_knitting.cutting instead.
  from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires


In [3]:
# useful additional packages
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

from qiskit_aer import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp 
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [6]:
def sprandsym(n, density,seed):
    np.random.seed((seed))
    rvs = stats.poisson(25, loc=10).rvs
    X = sparse.random(n, n, density=density, data_rvs=rvs)
    upper_X = sparse.triu(X) 
    result = upper_X + upper_X.T - sparse.diags(X.diagonal())
    return result

def binarize_sparse_matrix(sparse_matrix):
    # create a copy of the sparse matrix to keep the operation non-destructive
    sparse_copy = sparse_matrix.copy()
    #sparse_copy=sparse_copy-sparse.diags(sparse_copy.diagonal())
    # find the coordinates of non-zero elements
    non_zero_coords = sparse_copy.nonzero()
    # set those elements to 1
    sparse_copy[non_zero_coords] = 1
    return sparse_copy

def generate_graph_from_matrix(binarized_sparse_mat):
    G = nx.from_scipy_sparse_array(binarized_sparse_mat)
    return G


# create the quadratic program instance and define the variables
def create_qp_from_qmatrix(Q_matrix):
    max_keys = Q_matrix.shape[0]
    qp = QuadraticProgram('QUBO Matrix Optimization')
    x = qp.binary_var_list(name='x', keys=range(1, max_keys + 1))

    linear_vars = {qp.get_variable(i).name: Q_matrix[i, j]
                   for i in range(max_keys) for j in range(max_keys) if i == j}
    quadratic_vars = {(qp.get_variable(i).name, qp.get_variable(j).name): Q_matrix[i, j]
                      for i in range(max_keys) for j in range(max_keys) if i != j}

    qp.minimize(linear=linear_vars, quadratic=quadratic_vars)
    return qp
    #print(self.qp.prettyprint())


def create_qaoa_ansatz(qp):
    #self.create_qp_from_qmatrix()
    h_qubo, offset = qp.to_ising()
    #print(h_qubo)
    qaoa_ansatz = QAOAAnsatz(cost_operator=h_qubo, reps=1, )
    qaoa_ansatz.entanglement = 'linear'
    params = len(qaoa_ansatz.parameters)
    theta_range = np.linspace(0, np.pi, params)
    qaoa_qc = qaoa_ansatz.bind_parameters(theta_range)
    decomposed_qaoa_ansatz = qaoa_qc.decompose().decompose().decompose().decompose()
    return h_qubo, offset,decomposed_qaoa_ansatz


def get_subgraph_properties1(G):
    cnt=0
    subgraphs = (G.subgraph(c) for c in nx.connected_components(G))
    subgraph_prop = {}
    prop = []
    max_size = []
    max_subgraph_nodes = ''
    for s in subgraphs:
        #print(s.nodes())
        n = tuple(s.nodes())
        subgraph_prop[n] = nx.adjacency_matrix(s).todense()
        #print(s.size())
        #print(f'Subgraph {cnt}:: Num of Edges: {s.size()},  Nodes : {s.nodes()}  ')
        cnt+=1
        max_size.append(len(s.nodes()))
        if len(s.nodes)== np.max(max_size):
            max_subgraph_nodes = s.nodes()
        
        
    #print(max_subgraph_nodes)
    return cnt, np.max(max_size), subgraph_prop, max_subgraph_nodes




In [36]:
def cutqc(data, qaoa, n_qubits, num_subcircuits):
    print(f'inputs for cutqc : {n_qubits, num_subcircuits, qaoa}')
    error = False
    cuts = {}
    qubit_sc_idx = {}
    sc_idx = []

    cnt = 1
    try:
        while not cuts:
            print(f'cutting {cnt} time')
            cuts = cut_circuit_wires(
                circuit=qaoa,
                method="automatic",
                max_subcircuit_width=3,
                max_cuts=num_subcircuits+1,
                num_subcircuits=[num_subcircuits],
            )
            num_subcircuits+=1
    except Exception as e:
                print('in error')
                print(e)
                error = True
    
    if not error:
        #qubit-sc mappings from cutQC
        for scbit, scidx_arr in cuts['complete_path_map'].items():
            k = qaoa.find_bit(bit=scbit)[0]
            #k = scbit.index  # being deprecated
            qubit_sc_idx.setdefault(k,[])
            for scidx in range(len(scidx_arr)):
                sci = scidx_arr[scidx]['subcircuit_idx']
                qubit_sc_idx[k].append(sci)
                if sci not in sc_idx:
                    sc_idx.append(sci)

            '''
        Given the dictionary of qubit: subciruit_idx mapping, this function generates 
        adjacency matrices for the sub-circuits and cut positions required for build_cut_wire_circuit()
        '''

        sc_idx = set([v for i in qubit_sc_idx.values() for v in i ])
        sc_qubit_idx = {}
        for s in sc_idx:
            sc_qubit_idx.setdefault(s,[])
            for k, v in qubit_sc_idx.items():
                for j in v:
                    if s == j:
                        sc_qubit_idx[s].append(k)

        print(f'sc_qubit_idx: {sc_qubit_idx}') #subcircuit : qubit mapping 

        cut_pos = {}
        for k, v in qubit_sc_idx.items():
            # v is the set of sub-circuits 'k' qubit appears. 
            # if v longer than 1, there are more sub-circuits the qubit belongs to
            if len(v) > 1:
                cut_pos.setdefault(k,[])
                for l in range(len(v)-1):
                    #cut wire position for 'k' qubit - after which sub-circuit should the cut wire be applied
                    cut_pos[k].append(v[l])

        #cut wire position for 'k' qubit - after which sub-circuit should the cut wire be applied
        #eg: cut_pos = {0: [0], 3: [0]} - cut wires to be applied after 0th sub-circuit on qubit 0 and 
        # after 0th sub-circuit on 3rd qubit
        print(f'cut_pos: {cut_pos}')


        qaoa_sc = []
        for sc, q_idx in sc_qubit_idx.items():
            d = data.copy()
            for i in qubit_sc_idx.keys():
                if i not in q_idx:
                    d[:,i]=0
                    d[i,:]=0
            #print(d)
            qaoa_sc.append(d)

        print(f'qaoa_sc: {qaoa_sc}')
    
    return qaoa_sc, cut_pos


def build_cut_wire_circuit(qc, qaoa_sc, cut_pos):
    for idx, q in enumerate(qaoa_sc):
        qp1 = create_qp_from_qmatrix(q)
        print(qp1)
        h_qubo1, offset1, qaoa1 = create_qaoa_ansatz(qp1)
        print(h_qubo1)
        qc = qc.compose(qaoa1)

        #check if the sub-circuit idx in the cut pos
        for qubit_idx, subcircuit_idx in cut_pos.items():
            print(subcircuit_idx)
            if idx in subcircuit_idx:
                print('here')
                qc.append(CutWire(), [qubit_idx])
    return qc



def wirecutting(qc, max_cluster_size, qsubgraph_prop, wirecut_method):
    wm_qc = {}
    
    for wm_ in wirecut_method:
        qc_0 = qc
        for i, key in enumerate(qsubgraph_prop.keys()):  
            print(f'Subgraph nodes : {key}')
            data = qsubgraph_prop[key]
            q1 = nx.from_numpy_array(data)
            data = np.array(data)
            num_subcircuits = math.ceil(len(key)/max_cluster_size)

            qp = create_qp_from_qmatrix(data)
            h_qubit, offset, qaoa = create_qaoa_ansatz(qp)
            
            # if the fragment is bigger than n_qubits, apply cutqc-> qaoas for further fragments, cut positions. 
            #else, create qaoa and also cut position 
            if len(key)>max_cluster_size:
                
                #call wire cutting methods here
                if 'cut_qc' in wm_:
                    #qaoa_cutpos = {}
                    
   
                    qaoa_sc, cut_pos = cutqc(data, qaoa, max_cluster_size, num_subcircuits)
                    qc_0 = build_cut_wire_circuit(qc_0, qaoa_sc, cut_pos)

            else:
                qc_0 = qc_0.compose(qaoa)
        
        wm_qc[wm_]= qc_0
            
    return wm_qc


#qaoa_sc, cut_pos = find_subqaoa_cutpos(qubit_sc_idx)
#qc = QuantumCircuit(len(qubit_sc_idx.keys()))
#qc_0 = build_cut_wire_circuit(qc, qaoa_sc, cut_pos)

In [37]:
def ckt(qc_0, qc_1, observables):
    observables_1 = expand_observables(observables, qc_0, qc_1)
    partitioned_problem = partition_problem(circuit=qc_1, observables=observables_1)
    subcircuits = partitioned_problem.subcircuits
    subobservables = partitioned_problem.subobservables
    bases = partitioned_problem.bases
    sampling_overhead = np.prod([basis.overhead for basis in bases])
    return sampling_overhead
   

In [38]:
def ckt_exp(mat_size, prob, random_seeds, max_cluster_sizes):
    
    cols = ['n', 'p', 'seed', 'max_cluster_size', 'qsubgraph_prop']
    
    df = pd.DataFrame(columns=cols)
    i = 0
    for n in mat_size:
        for p in (prob): 
            for seed in random_seeds:
                print(f'\n\nExperiment for size {n}, density {p}, seed {seed}')
                
                ## Create sparse matrix for a given size and density
                M=sprandsym(n,p,seed)
                M=binarize_sparse_matrix(M)
                q=generate_graph_from_matrix(M)
                #q=generate_er_graph(n,p,seed)
                qnum_sub_graphs, largest_subgraph_size, qsubgraph_prop, max_subgraph_nodes = get_subgraph_properties1(q)
               
                ## Convert the sparse matrix into QUBO
                qp = create_qp_from_qmatrix(M)
                qp2qubo = QuadraticProgramToQubo()
                qubo = qp2qubo.convert(qp)
                qubitOp, offset = qubo.to_ising()
                
                ## Generate QAOA ansatz for QUBO and assign its observables
                qaoa = QAOAAnsatz(cost_operator=qubitOp,reps=1)
                qaoa_decomposed = qaoa.decompose().decompose().decompose().decompose()
                qaoa_observable_pat = '[A-Z]+'
                observables = PauliList(re.findall(qaoa_observable_pat, str(qubitOp)))
                
                ## Create the empty circuit 
                qc_0 = QuantumCircuit(n)
                
                ## Wire cutting methods for qaoa ansatz partitioning
                wirecut_methods = ['cut_qc']
                    

                ## Generate partition labels for the QAOA
                for max_cluster_size in max_cluster_sizes:
                    i += 1
                    wm_qc = wirecutting(qc_0, max_cluster_size, qsubgraph_prop, wirecut_methods)
                    
                    df.loc[i,'n'] = n
                    df.loc[i,'p'] = p
                    df.loc[i,'seed'] = seed
                    df.loc[i,'max_cluster_size'] = max_cluster_size
                    df.loc[i,'qsubgraph_prop'] = [qsubgraph_prop]
                    
                    for wm_ in wirecut_methods:
                        sampling_overhead = ckt(qc_0, wm_qc[wm_], observables)
                        df.loc[i,wm_] = sampling_overhead
                    
    return df
  

In [45]:
matrix_sizes =  [10] 
matrix_densities = [0.2]#,0.07,0.1,0.2,0.25,0.27] #[0.20,0.30,0.40,0.50,0.60,0.70]
num_of_experiments = 1
random_seeds = [random.randint(300, 1000) for _ in range(num_of_experiments)]
max_cluster_sizes = [3]#,5,7]

n=matrix_sizes[0]
p=matrix_densities[0]
seed = [random.randint(300, 1000) for _ in range(num_of_experiments)][0]

max_cluster_size = 3

In [46]:
df = ckt_exp(matrix_sizes, matrix_densities, random_seeds, max_cluster_sizes)



Experiment for size 10, density 0.2, seed 892
Subgraph nodes : (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
inputs for cutqc : (3, 4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f9f60c6afb0>)
cutting 1 time
4 subcircuits : IMPOSSIBLE
cutting 1 time
Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 8 times.
MIP Presolve eliminated 385 rows and 118 columns.
MIP Presolve modified 88 coefficients.
Aggregator did 367 substitutions.
Reduced MIP has 1340 rows, 500 columns, and 3974 nonzeros.
Reduced MIP has 485 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (18.89 ticks)
Probing fixed 38 vars, tightened 0 bounds.
Probing changed sense of 14 constraints.
Probing time = 0.01 sec. (5.12 ticks)
Cover probing fixed 0 vars, tightened 5 bounds.
Tried aggr

UnboundLocalError: local variable 'qaoa_sc' referenced before assignment

In [ ]:
df

In [ ]:
M=sprandsym(n,p,101)
M=binarize_sparse_matrix(M)
q=generate_graph_from_matrix(M)
qnum_sub_graphs, la`rgest_subgraph_size, qsubgraph_prop, max_subgraph_nodes = get_subgraph_properties1(q)


In [ ]:
data = qsubgraph_prop[tuple(max_subgraph_nodes)]
q1=nx.from_numpy_array(data)
node_lbl = dict(zip(list(q1.nodes), list(max_subgraph_nodes)))
nx.set_node_attributes(q1, values=node_lbl, name='labels')

In [ ]:
n_clusters = int(np.ceil(max_subgraph_size/max_cluster_size))
data = np.array(data)
qp = create_qp_from_qmatrix(data)
h_qubit, offset, qaoa = create_qaoa_ansatz(qp)

qaoa_observable_pat = '[A-Z]+'
observables = PauliList(re.findall(qaoa_observable_pat, str(h_qubit)))


In [ ]:
num_subcircuits = math.ceil(size/n_qubits)
cuts = cut_circuit_wires(
                        circuit=qaoa,
                        method="automatic",
                        max_subcircuit_width=n_qubits,
                        max_cuts= num_subcircuits+1,
                        num_subcircuits=[3]
                    )

In [ ]:
#qubit-sc mappings from cutQC
qubit_sc_idx = {}
sc_idx = []
for scbit, scidx_arr in cuts['complete_path_map'].items():
    k = qaoa.find_bit(bit=scbit)[0]
    #k = scbit.index  # being deprecated
    qubit_sc_idx.setdefault(k,[])
    for scidx in range(len(scidx_arr)):
        sci = scidx_arr[scidx]['subcircuit_idx']
        qubit_sc_idx[k].append(sci)
        if sci not in sc_idx:
            sc_idx.append(sci)

print(f'qubit_sc_idx : {qubit_sc_idx}') # qubit : subcircuit mapping
print(f'sc_idx: {sc_idx}')  # array of unique subcircuit indices  # not required 


def find_subqaoa_cutpos(qubit_sc_idx):
    '''
    Given the dictionary of qubit: subciruit_idx mapping, this function generates 
    adjacency matrices for the sub-circuits and cut positions required for build_cut_wire_circuit()
    '''
    
    sc_idx = set([v for i in qubit_sc_idx.values() for v in i ])
    sc_qubit_idx = {}
    for s in sc_idx:
        sc_qubit_idx.setdefault(s,[])
        for k, v in qubit_sc_idx.items():
            for j in v:
                if s == j:
                    sc_qubit_idx[s].append(k)

    print(f'sc_qubit_idx: {sc_qubit_idx}') #subcircuit : qubit mapping 

    cut_pos = {}
    for k, v in qubit_sc_idx.items():
        # v is the set of sub-circuits 'k' qubit appears. 
        # if v longer than 1, there are more sub-circuits the qubit belongs to
        if len(v) > 1:
            cut_pos.setdefault(k,[])
            for l in range(len(v)-1):
                #cut wire position for 'k' qubit - after which sub-circuit should the cut wire be applied
                cut_pos[k].append(v[l])

    #cut wire position for 'k' qubit - after which sub-circuit should the cut wire be applied
    #eg: cut_pos = {0: [0], 3: [0]} - cut wires to be applied after 0th sub-circuit on qubit 0 and 
    # after 0th sub-circuit on 3rd qubit
    print(f'cut_pos: {cut_pos}')


    qaoa_sc = []
    for sc, q_idx in sc_qubit_idx.items():
        d = data.copy()
        for i in qubit_sc_idx.keys():
            if i not in q_idx:
                d[:,i]=0
                d[i,:]=0
        #print(d)
        qaoa_sc.append(d)

    print(f'qaoa_sc: {qaoa_sc}')
    
    return qaoa_sc, cut_pos


def build_cut_wire_circuit(qc, qaoa_sc, cut_pos):
    for idx, q in enumerate(qaoa_sc):
        qp1 = create_qp_from_qmatrix(q)
        print(qp1)
        h_qubo1, offset1, qaoa1 = create_qaoa_ansatz(qp1)
        print(h_qubo1)
        qc = qc.compose(qaoa1)

        #check if the sub-circuit idx in the cut pos
        for qubit_idx, subcircuit_idx in cut_pos.items():
            print(subcircuit_idx)
            if idx in subcircuit_idx:
                print('here')
                qc.append(CutWire(), [qubit_idx])
    return qc

qaoa_sc, cut_pos = find_subqaoa_cutpos(qubit_sc_idx)
qc = QuantumCircuit(len(qubit_sc_idx.keys()))
qc_0 = build_cut_wire_circuit(qc, qaoa_sc, cut_pos)

In [ ]:
observables_1 = expand_observables(observables, qc_0, qc_1)
observables_1

In [ ]:
partitioned_problem = partition_problem(circuit=qc_1, observables=observables_1)
subcircuits = partitioned_problem.subcircuits
subobservables = partitioned_problem.subobservables

In [ ]:
bases = partitioned_problem.bases
print(f"Sampling overhead: {np.prod([basis.overhead for basis in bases])}")